In [3]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import geoip2.database
import myLogReader as mlr
import re
import os
import sys
import datetime as dt

%matplotlib inline

## Automate - Read and Prep log data into DF

In [4]:
logsPath = '../data/logs'
geoLiteIPDBPath = '../data/GeoLite2-City_20181009/GeoLite2-City.mmdb'

#Create a myLogReader object
myLogReader = mlr.log()
#Open Reader
myLogReader.openReader(geoLiteIPDBPath)

In [5]:
df =  myLogReader.readLogs(logsPath,1)

../data/logs\u_ex171213.log


In [7]:
#Close Reader
myLogReader.closeReader()

In [8]:
df

,date,cs-username,client-ip,time-taken(ms),client-device,client-browser,client-webPage,client-city,client-country,date-IsWeekday,...,time-taken(ms),client-device,client-browser,client-webPage,client-city,client-country,date-IsWeekday,date-calendar-week,date-year,calendar-year-week
0,2017-12-13,Unknown,50.112.95.211,312.0,Desktop,Other,Unknown,Boardman,United States,1,...,312.0,Desktop,Other,Unknown,Boardman,United States,1,50,2017,2017-50
1,2017-12-13,s423215,137.92.20.72,218.0,Desktop,Other,Maintenance,Canberra,Australia,1,...,218.0,Desktop,Other,Maintenance,Canberra,Australia,1,50,2017,2017-50
2,2017-12-13,Unknown,203.219.84.32,31.0,Mobile,Safari,Unknown,Canberra,Australia,1,...,31.0,Mobile,Safari,Unknown,Canberra,Australia,1,50,2017,2017-50
3,2017-12-13,Unknown,203.219.84.32,78.0,Mobile,Safari,Unknown,Canberra,Australia,1,...,78.0,Mobile,Safari,Unknown,Canberra,Australia,1,50,2017,2017-50
4,2017-12-13,Unknown,203.219.84.32,234.0,Mobile,Safari,Unknown,Canberra,Australia,1,...,234.0,Mobile,Safari,Unknown,Canberra,Australia,1,50,2017,2017-50
5,2017-12-13,Unknown,203.219.84.32,125.0,Mobile,Safari,Unknown,Canberra,Australia,1,...,125.0,Mobile,Safari,Unknown,Canberra,Australia,1,50,2017,2017-50
6,2017-12-13,Unknown,203.219.84.32,156.0,Mobile,Safari,Security,Canberra,Australia,1,...,156.0,Mobile,Safari,Security,Canberra,Australia,1,50,2017,2017-50
7,2017-12-13,Unknown,203.219.84.32,46.0,Mobile,Safari,Security,Canberra,Australia,1,...,46.0,Mobile,Safari,Security,Canberra,Australia,1,50,2017,2017-50
8,2017-12-13,Unknown,203.219.84.32,125.0,Mobile,Safari,Security,Canberra,Australia,1,...,125.0,Mobile,Safari,Security,Canberra,Australia,1,50,2017,2017-50
9,2017-12-13,Unknown,203.219.84.32,109.0,Mobile,Safari,Security,Canberra,Australia,1,...,109.0,Mobile,Safari,Security,Canberra,Australia,1,50,2017,2017-50


#### 2- Automate load logs and aggregate data

In [ ]:
#df.to_csv('test.csv')

In [ ]:
df_uniqueIP=(df.groupby(by=['calendar-year-week'])
            .agg({'client-ip': pd.Series.nunique})
            .rename(columns = {'client-ip':'client-ip-unique-count'}))

df_uniqueUsername=(df.groupby(by=['calendar-year-week'])
            .agg({'cs-username': pd.Series.nunique})
            .rename(columns = {'cs-username':'cs-username-unique-count'}))

df_totalconnections_timetaken= (df.groupby(by=['calendar-year-week'])
            .agg({'client-ip': pd.Series.count,'time-taken(ms)' : pd.Series.sum})
            .rename(columns = {'client-ip':'client-connections-count','time-taken(ms)' :'time-taken(ms)-sum' }))


df_browsercount=(df.groupby(by=['calendar-year-week','client-browser'])
            .agg({'client-browser': pd.Series.count})
            .rename(columns = {'client-browser':'client-browser-count'})      
            .reset_index(level=1)
            .pivot(columns='client-browser',values='client-browser-count')
            .rename(columns = {'Chrome':'Chrome-count'
                              ,'Firefox':'Firefox-count'
                              ,'Other':'Other-count'
                              ,'Safari':'Safari-count'}))



df_devicecount=(df.groupby(by=['calendar-year-week','client-device'])
            .agg({'client-device': pd.Series.count})  
            .rename(columns = {'client-device':'client-device-count'})      
            .reset_index(level=1)
            .pivot(columns='client-device',values='client-device-count')
            .rename(columns = {'Desktop':'Desktop-count'
                              ,'Mobile':'Mobile-count'}))

In [ ]:
pd.concat([df_devicecount,df_browsercount,df_totalconnections_timetaken,df_uniqueUsername,df_uniqueIP],axis=1)

In [ ]:
df.info()

In [ ]:
df['client-ip'].describe()

In [ ]:
df.loc[:, df.dtypes == object].describe()

In [ ]:
df.isna().sum()